# Single Path One-Shot Neural Architecture Search using Random Search

### Make everything a bit faster

Your task is to implement SPOS + Random Search to find the optimal ResNet-18-like architecture
for image classification on CIFAR-10 dataset.

### Proposed search space
The same search space as in the lecture, but without kernel size search.
This means that each search block should only have 3 operations.

### Latency awareness
While searching, you should implement hard constraint on model latency. As a proxy for
latency, use the number of MACs. The baseline ResNet-18 has 37M MACs for images of size 32x32.
The final selected architecture should have at most 30M MACs.
You can find an example computation for the number of MACs in this notebook.

### How to complete the task
Plan for your experiments:
1. Train baseline ResNet-18 model.  **(2 pts)**
2. Finalize supernet code. **(15 pts)**
3. Train supernet with the same hyperparameters except the number of epochs. You should increase
   it by a factor of 3-6. Save supernet weights.  **(10 pts)**
4. Write implementation for Random Search.  **(10 pts)**
5. Run it for 100-1000 iterations by measuring accuracies of models from the supernet.
   Keep track of each model accuracy and latency.  **(5 pts)**
6. Train the best architecture from scratch. For this, you can build the whole supernet, and
   select this architecture for all the forward passes of the training.  **(6 pts)**
7. Compare with the model from step 1. Does your model have a better accuracy?  **(2 pts)**

### Code structure

Main.ipynb - Learn your neural networks here.

resnet.py - ResNet-18 implementation from torchvision, simplified for this project.

supernet.py - Unfinished Supernet implementation.

cifar10.py - Transforms for CIFAR-10 training.


## Setup

Before you begin, make sure to install torch, torchvision, numpy, thop and tqdm libraries.

In [1]:
!pip install torch torchvision numpy thop tqdm

Defaulting to user installation because normal site-packages is not writeable


### Imports

In [33]:
from typing import Tuple

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from thop import profile
from torch.utils.data import DataLoader
from tqdm import tqdm

from cifar10 import get_train_transform, get_val_transform
from resnet import resnet18

### Make everything a bit faster

In [34]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

### Build datasets and dataloaders for CIFAR-10

In [35]:
# Change this value if needed.
batch_size = 512

In [36]:
train_transform = get_train_transform()
val_transform = get_val_transform()

train_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=train_transform,
)
test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=val_transform,
)

train_dataloader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    drop_last=True,
)
test_dataloader = DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    drop_last=False,
)

Files already downloaded and verified
Files already downloaded and verified


## Create your baseline model

In [37]:
# Select suitable device.
# You should probably use either cuda (NVidia GPU) or mps (Apple) backend.
device = torch.device('cuda:0')

In [15]:
model = resnet18(num_classes=10, zero_init_residual=True)
model.to(device=device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Compute the number of MACs and parameters for the model

In [8]:
macs, params = profile(model, inputs=(torch.zeros(1, 3, 32, 32, device=device),))
print(f'Number of macs: {macs / 1e6:.2f}M, number of parameters: {params / 1e6:.2f}M')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Number of macs: 37.22M, number of parameters: 11.18M


## Train baseline model

### Define loss function

In [38]:
criterion = nn.CrossEntropyLoss()

### Select hyperparameters

In [39]:
lr = 0.25
weight_decay = 5e-4
momentum = 0.9
n_epochs = 20  # Longer training gives better results, but let's keep baseline model epochs to 20.

### Build optimizer and scheduler

In [40]:
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader) * n_epochs)

### Define training and evaluation functions

In [42]:
def train_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        optimizer: optim.Optimizer,
        scheduler,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.train()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in wrapped_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        optimizer.zero_grad()

        logits = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        with torch.no_grad():
            _, predicted_labels = torch.max(logits, 1)
            total_loss += loss.item()
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.shape[0]

        wrapped_dataloader.set_description(
            f'(train) Epoch={epoch}, lr={scheduler.get_last_lr()[0]:.4f} loss={total_loss / (i + 1):.3f}'
        )

    return total_loss / len(dataloader), total_correct / total_samples


@torch.no_grad()
def validate_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.eval()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in wrapped_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        logits = model(inputs)
        loss = criterion(logits, labels)
        _, predicted_labels = torch.max(logits, 1)
        total_loss += loss.item()
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.shape[0]

        wrapped_dataloader.set_description(f'(val) Epoch={epoch}, loss={total_loss / (i + 1):.3f}')

    return total_loss / len(dataloader), total_correct / total_samples

### Run training

In [13]:
for epoch in range(n_epochs):
    print(f'Epoch: {epoch}')
    loss, accuracy = train_one_epoch(model, criterion, train_dataloader, optimizer, scheduler, device, epoch)
    print(f'train_loss={loss:.4f}, train_accuracy={accuracy:.3%}')
    loss, accuracy = validate_one_epoch(model, criterion, test_dataloader, device, epoch)
    print(f'test_loss={loss:.4f}, test_accuracy={accuracy:.3%}')

Epoch: 0


(train) Epoch=0, lr=0.2485 loss=2.084: 100%|██████████| 97/97 [00:03<00:00, 31.62it/s]


train_loss=2.0840, train_accuracy=27.136%


(val) Epoch=0, loss=1.501: 100%|██████████| 20/20 [00:00<00:00, 49.73it/s]

test_loss=1.5008, test_accuracy=45.030%
Epoch: 1


...

test_loss=0.5009, test_accuracy=83.030%
Epoch: 19



(train) Epoch=19, lr=0.0000 loss=0.403: 100%|██████████| 97/97 [00:02<00:00, 35.12it/s]

train_loss=0.4032, train_accuracy=86.032%



(val) Epoch=19, loss=0.501: 100%|██████████| 20/20 [00:00<00:00, 63.74it/s]

test_loss=0.5006, test_accuracy=83.060%


### Save trained model weights

In [14]:
torch.save(model.state_dict(), 'baseline_model.pth')

## Neural Architecture Search - Supernet training

### Create supernet

**Before running code in this section, you need to finish supernet implementation.**

Please, go to `supernet.py` file and inspect the current implementation of SearchBlock and Supernet classes.
Pay attention to the TODOs. You need to implement all of them.

Supernet and BasicBlock classes are modified versions of ResNet and BasicBlock classes from `resnet.py`.

    Tip: to understand how the Supernet is constructed, compare the implementation of Supernet and ResNet classes. You should probably use diff tool in your IDE or something.

Task: briefly describe the differences made to construct supernet.

Differences:
1. Bottleneck block is replaced by SearchBlock
2. Supernet is designed for search the optimal SearchBlock structure per layer.

In [23]:
from supernet import supernet18

In [24]:
# TODO: Define inner channel multipliers as in lecture.
channel_multipliers = (0.5, 0.75, 1.0)

In [25]:
supernet = supernet18(num_classes=10, zero_init_residual=True, channel_multipliers=channel_multipliers)
supernet.to(device=device)

Supernet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): SearchBlock(
      (ops): ModuleList(
        (0): BasicBlock(
          (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(48, eps=1e-05,

### Select hyperparameters

In [26]:
# TODO: define hyperparameters for supernet training.
criterion = nn.CrossEntropyLoss()
lr = 0.25
weight_decay = 5e-4
momentum = 0.9
n_epochs = 100  # Longer training gives better results, but let's keep baseline model epochs to 20.

### Build optimizer and scheduler

In [27]:
# TODO: build optimizer and scheduler for supernet training.
optimizer = optim.SGD(supernet.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader) * n_epochs)

### Define training function

In [28]:
# TODO: Copy `train_one_epoch` function here, rename to `pretrain_one_epoch`.
#       Call `model.sample_random_architecture()` before making forward pass on each batch.
def pretrain_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        optimizer: optim.Optimizer,
        scheduler,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.train()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    #wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        optimizer.zero_grad()
        model.sample_random_architecture()
        logits = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        with torch.no_grad():
            _, predicted_labels = torch.max(logits, 1)
            total_loss += loss.item()
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.shape[0]

        # wrapped_dataloader.set_description(
        #     f'(train) Epoch={epoch}, lr={scheduler.get_last_lr()[0]:.4f} loss={total_loss / (i + 1):.3f}'
        # )

    return total_loss / len(dataloader), total_correct / total_samples


@torch.no_grad()
def validate_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.eval()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    #wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        logits = model(inputs)
        loss = criterion(logits, labels)
        _, predicted_labels = torch.max(logits, 1)
        total_loss += loss.item()
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.shape[0]

        #wrapped_dataloader.set_description(f'(val) Epoch={epoch}, loss={total_loss / (i + 1):.3f}')

    return total_loss / len(dataloader), total_correct / total_samples

In [29]:
for epoch in tqdm(range(n_epochs)):
    loss, accuracy = pretrain_one_epoch(supernet, criterion, train_dataloader, optimizer, scheduler, device, epoch)
    if epoch % 20 ==0:
        print(f'Epoch: {epoch}')
        print(f'train_loss={loss:.4f}, train_accuracy={accuracy:.3%}')
    loss, accuracy = validate_one_epoch(supernet, criterion, test_dataloader, device, epoch)
    if epoch % 20 ==0:
        print(f'test_loss={loss:.4f}, test_accuracy={accuracy:.3%}')

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0
train_loss=4.2950, train_accuracy=10.710%


  1%|          | 1/100 [00:02<04:09,  2.52s/it]

test_loss=5.3872, test_accuracy=8.600%


 20%|██        | 20/100 [00:49<03:14,  2.43s/it]

...

 80%|████████  | 80/100 [03:46<01:05,  3.29s/it]

Epoch: 80
train_loss=0.4646, train_accuracy=83.743%


 81%|████████  | 81/100 [03:50<01:03,  3.34s/it]

test_loss=0.5290, test_accuracy=82.080%


100%|██████████| 100/100 [04:52<00:00,  2.93s/it]


### Save supernet weights

In [ ]:
torch.save(supernet.state_dict(), 'supernet.pth')

## Neural Architecture Search - Random Search.

In [30]:
import time

def measure_latency(model: torch.nn.Module, device: torch.device, input_shape=(1, 3, 224, 224), dtype=torch.float32, warmup_iters=10, measure_iters=100) -> float:
    """
    Measure the latency of a forward pass in a PyTorch model.

    Args:
    - model: The PyTorch model to measure.
    - device: The device to run the measurement on (e.g., 'cuda:0', 'cpu').
    - input_shape: The shape of the dummy input tensor for the model.
    - dtype: The data type of the dummy input tensor.
    - warmup_iters: Number of iterations to warm up the model (important for GPUs).
    - measure_iters: Number of iterations to measure the latency.

    Returns:
    - The average latency in milliseconds of a single forward pass.
    """
    
    # Move model to the specified device
    model.to(device)
    
    # Generate a dummy input tensor
    dummy_input = torch.randn(input_shape, dtype=dtype, device=device)
    
    # Warm-up phase
    with torch.no_grad():
        for _ in range(warmup_iters):
            _ = model(dummy_input)
    
    # Measurement phase
    start_time = time.time()
    with torch.no_grad():
        for _ in range(measure_iters):
            _ = model(dummy_input)
    end_time = time.time()
    
    # Calculate average latency
    avg_latency = (end_time - start_time) / measure_iters * 1000.0  # Convert to milliseconds
    
    return avg_latency

In [31]:
import torch
import numpy as np
from typing import Union, List
from torch.utils.data import DataLoader
from tqdm import tqdm

def random_search(
        trained_supernet: nn.Module,
        train_dataloader: DataLoader,
        val_dataloader: DataLoader,
        device: torch.device,
        n_architectures_to_test: int,
        target_latency: float,) -> Union[float, List[float]]:
    # TODO: Implement Random search.
    # This function should evaluate `n_architectures_to_test` architectures
    # using the `trained_supernet` on the whole validation dataset. The resulting
    # architecture should have latency not greater than `target_latency`.
    # Rank architectures by validation accuracy.
    best_accuracy = 0.0
    best_architecture = []
    
    for _ in tqdm(range(n_architectures_to_test)):
        # Sample a random architecture
        trained_supernet.sample_random_architecture()
        current_architecture = [block.active_op_index for block in trained_supernet.search_blocks]
        
        # Measure its latency
        current_latency = measure_latency(trained_supernet, device)
        
        # Skip this architecture if it exceeds the target latency
        if current_latency > target_latency:
            continue
        
        # Evaluate the architecture
        trained_supernet.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = trained_supernet(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = correct / total
        
        # Update best architecture if this one is better
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_architecture = current_architecture
    
    return best_accuracy, best_architecture

In [32]:
n_architectures_to_test = 100
target_latency = 20 * 1e6

accuracy, best_architecture = random_search(supernet, train_dataloader, test_dataloader, device, n_architectures_to_test, target_latency)
print(f'best architecture: {best_architecture} (test_accuracy={accuracy:.3%})')

100%|██████████| 100/100 [00:44<00:00,  2.25it/s]

best architecture: [0, 2, 2, 2, 2, 0, 1, 1] (test_accuracy=83.930%)


## Train found architecture from scratch

Go to `Create your baseline model` section.

Change model definition to this to build unitialized best architecture from the search space:

```python
model = supernet18(num_classes=10, zero_init_residual=True, channel_multipliers=channel_multipliers)
model.to(device=device)
model.sample(best_architecture)
```

After this, simply run all the cells in that section.

In [43]:

best_architecture = [2, 2, 1, 2, 1, 0, 1, 2]
model = supernet18(num_classes=10, zero_init_residual=True, channel_multipliers=channel_multipliers)
model.to(device=device)
model.sample(best_architecture)

macs, params = profile(model, inputs=(torch.zeros(1, 3, 32, 32, device=device),))
print(f'Number of macs: {macs / 1e6:.2f}M, number of parameters: {params / 1e6:.2f}M')

for epoch in range(n_epochs):
    print(f'Epoch: {epoch}')
    loss, accuracy = train_one_epoch(model, criterion, train_dataloader, optimizer, scheduler, device, epoch)
    print(f'train_loss={loss:.4f}, train_accuracy={accuracy:.3%}')
    loss, accuracy = validate_one_epoch(model, criterion, test_dataloader, device, epoch)
    print(f'test_loss={loss:.4f}, test_accuracy={accuracy:.3%}')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Number of macs: 32.20M, number of parameters: 9.43M
Epoch: 0


(train) Epoch=0, lr=0.2485 loss=2.380: 100%|██████████| 97/97 [00:02<00:00, 45.69it/s]

train_loss=2.3804, train_accuracy=9.200%



(val) Epoch=0, loss=2.386: 100%|██████████| 20/20 [00:00<00:00, 67.77it/s]

test_loss=2.3862, test_accuracy=8.120%
Epoch: 1


...

test_loss=2.3872, test_accuracy=8.040%
Epoch: 19



(train) Epoch=19, lr=0.0000 loss=2.377: 100%|██████████| 97/97 [00:02<00:00, 42.79it/s]

train_loss=2.3773, train_accuracy=9.228%



(val) Epoch=19, loss=2.386: 100%|██████████| 20/20 [00:00<00:00, 64.18it/s]

test_loss=2.3863, test_accuracy=8.120%
